# Job Classifier LLM
This notebook was running on Google Colab with GPU

In [ ]:
!pip uninstall transformers -y
!pip install transformers --upgrade


Found existing installation: transformers 4.55.0
Uninstalling transformers-4.55.0:
  Successfully uninstalled transformers-4.55.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 57.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load Job Posting Dataset

In [ ]:
from datasets import load_dataset

job_dataset = load_dataset("json", data_files="dataset.json")
job_dataset.shape

In [ ]:
job_dataset = job_dataset['train'].train_test_split(test_size=0.2)
job_dataset

In [ ]:
def combine_text(example):
  return {"text": example['title']+" "+example['company_profile']+" "+example['description']+" "+example['requirements']+" "+example['benefits']}

In [ ]:
job_dataset = job_dataset.filter(lambda x: x['title'] and x['company_profile'] and x['description'] and x['requirements'] and x['benefits'])

In [ ]:
job_dataset = job_dataset.map(combine_text)
job_dataset

In [ ]:
job_dataset['train'][0]['text']

In [ ]:
job_dataset = job_dataset.rename_column(
    original_column_name="fraudulent", new_column_name="labels"
)
job_dataset

In [ ]:
job_dataset['train'].features

In [ ]:
import re
from bs4 import BeautifulSoup
import html

def clean_text(batch):
    cleaned_texts = []
    for t in batch["text"]:
        t = BeautifulSoup(t, "html.parser").get_text()
        t = html.unescape(t)
        t = re.sub(r"#URL_[^#]+#", "", t)
        t = re.sub(r"\s+", " ", t).strip()
        cleaned_texts.append(t)
    return {"text": cleaned_texts}

job_dataset = job_dataset.map(clean_text, batched=True)


In [ ]:
job_dataset['train'][0]['text']

In [ ]:
# split train to create validation

job_dataset_clean = job_dataset['train'].train_test_split(test_size=0.2, seed=42)
job_dataset_clean['validation'] = job_dataset_clean.pop('test')
job_dataset_clean['test'] = job_dataset['test']

job_dataset_clean

In [ ]:
for split, dataset in job_dataset_clean.items():
    dataset.to_json(f"job-post-{split}.jsonl")

Get Back Data

In [ ]:
data_files = {
    "train": "job-post-train.jsonl",
    "validation": "job-post-validation.jsonl",
    "test": "job-post-test.jsonl",
}
job_dataset = load_dataset("json", data_files=data_files)

job_dataset

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Tokenize

In [ ]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

In [ ]:
tokenized_dataset = job_dataset.map(
    tokenize_and_split, batched=True
)

In [ ]:
tokenized_dataset

Train Model with Only Text Features

In [ ]:
to_remove_cols = [col for col in job_dataset['train'].column_names if col not in ['labels', 'text']]
tokenized_dataset = tokenized_dataset.remove_columns(to_remove_cols)
tokenized_dataset


In [ ]:
tokenized_dataset['train'].features

Start Fine Tuning

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
id2label = {0: "REAL", 1: "FAKE"}
label2id = {"REAL": 0, "FAKE": 1}

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)
model.to("cuda")
print(torch.cuda.is_available())

In [ ]:
import evaluate
import numpy as np
from sklearn.metrics import matthews_corrcoef, balanced_accuracy_score, confusion_matrix

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
roc_auc_metric = evaluate.load("roc_auc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1_macro = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    prec_macro = precision_metric.compute(predictions=predictions, references=labels, average="macro")
    rec_macro = recall_metric.compute(predictions=predictions, references=labels, average="macro")

    mcc = matthews_corrcoef(labels, predictions)

    return {
        "accuracy": acc["accuracy"],
        "f1_macro": f1_macro["f1"],
        "precision_macro": prec_macro["precision"],
        "recall_macro": rec_macro["recall"],
        "mcc": mcc,
    }


In [ ]:
import torch

def compute_class_weights_from_counts(count_class0, count_class1):
    """
    Compute sklearn-style class weights given counts for each class.
    Args:
        count_class0 (int): Number of samples in class 0
        count_class1 (int): Number of samples in class 1
    Returns:
        torch.tensor of shape [2] with weights for [class0, class1]
    """
    total = count_class0 + count_class1
    num_classes = 2
    weights = [
        total / (num_classes * count_class0),  # weight for class 0
        total / (num_classes * count_class1),  # weight for class 1
    ]
    return torch.tensor(weights, dtype=torch.float)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    eval_strategy="epoch",
    logging_steps=50,
    save_steps=500,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
import torch
import torch.nn.functional as F
from collections import Counter
from transformers import Trainer

def focal_loss_with_class_weights(logits, labels, class_weights=None, gamma=2.0):
    """
    logits: [batch_size, num_classes]
    labels: [batch_size]
    class_weights: torch.tensor shape [num_classes]
    """
    device = logits.device  # get device from logits (cuda or cpu)
    if class_weights is not None:
        class_weights = class_weights.to(device)  # move class_weights to correct device

    ce_loss = F.cross_entropy(logits, labels, weight=class_weights, reduction="none")
    pt = torch.exp(-ce_loss)  # probability of true class
    focal_term = (1 - pt) ** gamma
    return (focal_term * ce_loss).mean()

class CustomLossTrainer(Trainer):
    def __init__(self, *args, loss_fn=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fn = loss_fn

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss



weights = compute_class_weights_from_counts(17589, 855)
weights = torch.tensor(weights, dtype=torch.float)


trainer = CustomLossTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    loss_fn=lambda logits, labels: focal_loss_with_class_weights(
        logits, labels, class_weights=weights, gamma=2.0
    ),
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-3641210626.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  weights = torch.tensor(weights, dtype=torch.float)


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro,Mcc
1,0.181200,0.237680,0.958738,0.766675,0.743908,0.795031,0.536509
2,0.149200,0.296379,0.975625,0.829508,0.874892,0.794716,0.664791
3,0.064700,0.238815,0.978095,0.849723,0.888116,0.818820,0.703531


TrainOutput(global_step=19167, training_loss=0.2080575880797317, metrics={'train_runtime': 2702.1275, 'train_samples_per_second': 56.74, 'train_steps_per_second': 7.093, 'total_flos': 5077409156785152.0, 'train_loss': 0.2080575880797317, 'epoch': 3.0})